PyCity Schools Analysis Solution

In [17]:
import pandas as pd

# Cargar los archivos de datos
school_data = pd.read_csv("C:/Users/maria.hernandezpauli/pandas-challenge/PyCitySchools/Resources/schools_complete.csv")
student_data = pd.read_csv("C:/Users/maria.hernandezpauli/pandas-challenge/PyCitySchools/Resources/students_complete.csv")

# Combinar los dos DataFrames en uno solo
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name"])

# Total de escuelas únicas
total_schools = school_data["school_name"].nunique()
print(total_schools)

# Total de estudiantes
total_students = student_data["student_name"].count()
print(total_students)

# Presupuesto total
total_budget = school_data["budget"].sum()
print(total_budget)

# Promedio de calificaciones en matemáticas y lectura
average_math_score = student_data["math_score"].mean()
average_reading_score = student_data["reading_score"].mean()
print(average_math_score)
print(average_reading_score)

# Porcentaje de estudiantes que aprobaron matemáticas y lectura
passing_math = (student_data["math_score"] >= 70).mean() * 100
passing_reading = (student_data["reading_score"] >= 70).mean() * 100
print(passing_math)
print(passing_reading)

# Porcentaje general de aprobación
overall_passing = ((student_data["math_score"] >= 70) & (student_data["reading_score"] >= 70)).mean() * 100
print(overall_passing)

# Crear el DataFrame del resumen del distrito
district_summary = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [passing_math],
    "% Passing Reading": [passing_reading],
    "% Overall Passing": [overall_passing]
})

# Redondear los puntajes al entero más cercano
district_summary["Average Math Score"] = district_summary["Average Math Score"].round(0)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].round(0)

# Formatear el presupuesto total como dinero
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Formatear los porcentajes con 2 decimales y el símbolo %
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.2f}%".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.2f}%".format)
district_summary["% Overall Passing"] = district_summary["% Overall Passing"].map("{:.2f}%".format)

# Mostrar el resumen del distrito
print(district_summary)

# Calcular las métricas clave para cada escuela
per_school_summary = school_data_complete.groupby("school_name").agg({
    "type": "first",
    "Student ID": "count",
    "budget": "first",
    "math_score": "mean",
    "reading_score": "mean"
})

# Renombrar las columnas para que sean más descriptivas
per_school_summary.columns = [
    "School Type",
    "Total Students",
    "Total School Budget",
    "Average Math Score",
    "Average Reading Score"
]

# Calcular el presupuesto por estudiante
per_school_summary["Per Student Budget"] = per_school_summary["Total School Budget"] / per_school_summary["Total Students"]

# Eliminar el formato de moneda y convertir a float para la columna "Per Student Budget"
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].replace({r'[$,]': ''}, regex=True).astype(float)

# Porcentaje de estudiantes que aprobaron matemáticas
passing_math_by_school = school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name")["Student ID"].count()
per_school_summary["% Passing Math"] = (passing_math_by_school / per_school_summary["Total Students"]) * 100

# Porcentaje de estudiantes que aprobaron lectura
passing_reading_by_school = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name")["Student ID"].count()
per_school_summary["% Passing Reading"] = (passing_reading_by_school / per_school_summary["Total Students"]) * 100

# Porcentaje de estudiantes que aprobaron ambas materias
overall_passing_by_school = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].groupby("school_name")["Student ID"].count()
per_school_summary["% Overall Passing"] = (overall_passing_by_school / per_school_summary["Total Students"]) * 100

# Redondear los puntajes al entero más cercano
per_school_summary["Average Math Score"] = per_school_summary["Average Math Score"].round(0)
per_school_summary["Average Reading Score"] = per_school_summary["Average Reading Score"].round(0)

# Crear los bins y etiquetar los rangos de gasto
spending_bins = [0, 585, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]
per_school_summary["Spending Ranges (Per Student)"] = pd.cut(per_school_summary["Per Student Budget"], bins=spending_bins, labels=spending_labels)

# Agrupar por rango de gasto y calcular las métricas, excluyendo columnas no numéricas
spending_summary = per_school_summary.groupby("Spending Ranges (Per Student)", observed=False).mean(numeric_only=True)[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing"
]]
print(spending_summary)

# Crear bins y etiquetas para el tamaño de la escuela
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], bins=size_bins, labels=size_labels)

# Agrupar por tamaño y calcular las métricas, excluyendo columnas no numéricas
size_summary = per_school_summary.groupby("School Size", observed=False).mean(numeric_only=True)[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing"
]]
print(size_summary)

# Agrupar por tipo de escuela y calcular las métricas, excluyendo columnas no numéricas
type_summary = per_school_summary.groupby("School Type", observed=False).mean(numeric_only=True)[[
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing"
]]
print(type_summary)


15
39170
24649428
78.98537145774827
81.87784018381414
74.9808526933878
85.80546336482001
65.17232575950983
   Total Schools  Total Students    Total Budget  Average Math Score  \
0             15           39170  $24,649,428.00                79.0   

   Average Reading Score % Passing Math % Passing Reading % Overall Passing  
0                   82.0         74.98%            85.81%            65.17%  
                               Average Math Score  Average Reading Score  \
Spending Ranges (Per Student)                                              
<$585                                       83.50                  84.00   
$585-630                                    81.75                  83.25   
$630-645                                    78.50                  81.75   
$645-680                                    77.00                  81.00   

                               % Passing Math  % Passing Reading  \
Spending Ranges (Per Student)                                      